# Objetivo

Aplicar o K Nearest Neighbors Na base toda. 

# Método

A partir do conjunto reduzido de features (39) 100k rows, aplicar:
- fillna <b> OK </b>

- LabelEncoder (ou algo do tipo) <b> OK </b>
- Nearest Neighbors OK
    

# Leitura dos arquivos

In [2]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

## Leitura da lista de features a excluir

# Carregamento Clientes

## Redução no número de features



In [3]:
market = pd.read_csv('../../estaticos_market.csv')
port1 = pd.read_csv('clientes2_merge.csv')

In [4]:
exclude_df = pd.read_csv('exclude_variables_generated_from_profiling.csv')
exclude_variables = exclude_df.values.squeeze().tolist()
len(exclude_variables)


143

In [5]:
port1.shape

(566, 182)

In [6]:
market.drop(columns=exclude_variables, inplace= True)
market.drop(columns=['Unnamed: 0'], inplace= True)
market.set_index('id',inplace=True)
market.shape

(462298, 37)

In [7]:
port1.drop(columns=exclude_variables, inplace= True)
port1.drop(columns=['Unnamed: 0'], inplace= True)
port1.set_index('id',inplace=True)
port1.shape

(566, 37)

In [7]:
#p1.to_csv('estaticos_49_features.csv')

# Managing NaNs


Para usar nos dados do portfólio:
- passar as strings para valor numérico
- passar bool para int
- fazer fillna para todos
- - em categoricas -> other
- - em numericas -> 0

Estamos usando o LabelEncoder

In [8]:
fill_dict = market.dtypes.to_dict()
cat2bool = ['fl_optante_simei','fl_optante_simples','fl_passivel_iss']

for feature in fill_dict:
    if(fill_dict[feature] == bool):
        fill_dict[feature] = False
    elif(fill_dict[feature] == object):
        fill_dict[feature] = 'other'
    else:
        fill_dict[feature] = 0
    if feature in cat2bool:
        fill_dict[feature] = False
        
fill_dict

{'fl_matriz': False,
 'de_natureza_juridica': 'other',
 'sg_uf': 'other',
 'natureza_juridica_macro': 'other',
 'de_ramo': 'other',
 'setor': 'other',
 'idade_empresa_anos': 0,
 'idade_emp_cat': 'other',
 'fl_me': False,
 'fl_sa': False,
 'fl_mei': False,
 'fl_ltda': False,
 'dt_situacao': 'other',
 'fl_st_especial': False,
 'fl_email': False,
 'fl_telefone': False,
 'fl_rm': 'other',
 'nm_divisao': 'other',
 'nm_segmento': 'other',
 'fl_optante_simples': False,
 'fl_optante_simei': False,
 'sg_uf_matriz': 'other',
 'de_saude_tributaria': 'other',
 'de_saude_rescencia': 'other',
 'nu_meses_rescencia': 0,
 'de_nivel_atividade': 'other',
 'empsetorcensitariofaixarendapopulacao': 0,
 'nm_meso_regiao': 'other',
 'nm_micro_regiao': 'other',
 'fl_passivel_iss': False,
 'qt_socios': 0,
 'idade_media_socios': 0,
 'qt_socios_feminino': 0,
 'de_faixa_faturamento_estimado': 'other',
 'de_faixa_faturamento_estimado_grupo': 'other',
 'vl_faturamento_estimado_aux': 0,
 'vl_faturamento_estimado_grupo

In [9]:
market.fillna(value=fill_dict, inplace=True)
market[cat2bool] = market[cat2bool].astype(bool)

bool2numeric = market.columns[market.dtypes == bool]
market[bool2numeric] = market[bool2numeric].astype(int)

In [11]:
#market_p1.head(2)

## Encoding

In [12]:
cat = market.columns[market.dtypes == object]
not_cat = market.columns[market.dtypes != object]

In [13]:
# Usando LabelEncoder
#https://chrisalbon.com/machine_learning/preprocessing_structured_data/convert_pandas_categorical_column_into_integers_for_scikit-learn/
from sklearn import preprocessing
# Create a label (category) encoder object
le = preprocessing.LabelEncoder()

In [14]:
market_le = pd.DataFrame()
market_le[cat] = market[cat].apply(lambda col: le.fit_transform(col))
market_encoded = pd.concat([market[not_cat],market_le], axis =1)
market_encoded.dtypes

fl_matriz                                  int64
idade_empresa_anos                       float64
fl_me                                      int64
fl_sa                                      int64
fl_mei                                     int64
fl_ltda                                    int64
fl_st_especial                             int64
fl_email                                   int64
fl_telefone                                int64
fl_optante_simples                         int64
fl_optante_simei                           int64
nu_meses_rescencia                       float64
empsetorcensitariofaixarendapopulacao    float64
fl_passivel_iss                            int64
qt_socios                                float64
idade_media_socios                       float64
qt_socios_feminino                       float64
vl_faturamento_estimado_aux              float64
vl_faturamento_estimado_grupo_aux        float64
de_natureza_juridica                       int64
sg_uf               

Aqui fiz fiz um novo dataset para os ids do portifolio1 e o merge com os dados já aplicados o LabelEncoder

In [15]:
port1_le = pd.DataFrame(index=port1.index)
port1_le = market_encoded.merge(port1_le, how='right',left_index=True,right_index=True)


In [16]:
port1_le.head(3)

,fl_matriz,idade_empresa_anos,fl_me,fl_sa,fl_mei,fl_ltda,fl_st_especial,fl_email,fl_telefone,fl_optante_simples,...,nm_divisao,nm_segmento,sg_uf_matriz,de_saude_tributaria,de_saude_rescencia,de_nivel_atividade,nm_meso_regiao,nm_micro_regiao,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo
id,,,,,,,,,,,,,,,,,,,,,
09e95c1a84049001d086470a8f320a19b076f955a89122ff8360ac2e4d6eca5d,1,11.243836,0,0,0,0,0,1,1,0,...,0,0,0,4,0,0,17,57,6,6
dc9d155f5bcd3172031e2ec9431786e8871ff4b0ff4728bb62194ca0842cce3e,1,43.789041,0,0,0,0,0,0,0,0,...,0,0,19,4,0,0,0,1,4,4
16843c9ffb920170477118933798510d8d5f00f5c56c1221773356355bded321,1,16.389041,0,0,0,0,0,0,1,0,...,36,11,16,4,0,0,9,11,3,3


# Nearest Neighbors em uma amostra do banco

Como não temos o que prever, usamos o treinamento não supervisionado do Nearest Neighbors



Apenas amostra por enquanto

Precisa escolher o numero de componentes a indicar

In [17]:
market_sample = market_encoded.sample(n=500)

In [18]:
from sklearn.neighbors import NearestNeighbors

nearest = NearestNeighbors(n_neighbors=2, metric='cosine',radius = 5)

nearest.fit(market_sample)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=2, p=2, radius=5)

Dict para a lista de vizinhos mais próximos

In [19]:
neighbors_list = {}

for row in range(port1_le.shape[0]):
    #print(row)
    neighbors_list[row] = nearest.kneighbors(port1_le.iloc[[row]].values)

A resposta é uma lista da distância e uma lista de componentes. A componente é a linha do Dataset, ou seja, clientes.

In [20]:
neighbors_list

{0: (array([[2.04664286e-10, 9.29182842e-10]]), array([[307, 426]])),
 1: (array([[1.97027838e-10, 8.11127387e-10]]), array([[307, 426]])),
 2: (array([[2.86514035e-10, 1.33795186e-09]]), array([[426, 307]])),
 3: (array([[2.05112594e-10, 8.61469340e-10]]), array([[307, 426]])),
 4: (array([[2.87267099e-10, 1.33573874e-09]]), array([[426, 307]])),
 5: (array([[3.67804442e-09, 4.59268623e-09]]), array([[184, 426]])),
 6: (array([[4.82346052e-10, 6.49955534e-10]]), array([[307, 426]])),
 7: (array([[1.71544684e-05, 9.29796931e-05]]), array([[330, 448]])),
 8: (array([[8.04776940e-06, 8.04877895e-06]]), array([[426, 307]])),
 9: (array([[5.59139846e-10, 1.99393035e-09]]), array([[426, 307]])),
 10: (array([[5.37392353e-11, 6.49186926e-10]]), array([[426, 307]])),
 11: (array([[2.76300129e-05, 8.14040290e-05]]), array([[280, 370]])),
 12: (array([[3.21775406e-09, 3.48489826e-09]]), array([[426, 184]])),
 13: (array([[1.48803114e-09, 2.00849259e-09]]), array([[426, 307]])),
 14: (array([[1.

## Avaliação

Agora com o NN funcionando, precisa confrontar as sugestões para saber qual é o mais sugerido.

## Verificando o que foi mais sugerido

Da lista de vizinhos, listar em ordem os mais frequentes.

In [21]:
neighbors_list

{0: (array([[2.04664286e-10, 9.29182842e-10]]), array([[307, 426]])),
 1: (array([[1.97027838e-10, 8.11127387e-10]]), array([[307, 426]])),
 2: (array([[2.86514035e-10, 1.33795186e-09]]), array([[426, 307]])),
 3: (array([[2.05112594e-10, 8.61469340e-10]]), array([[307, 426]])),
 4: (array([[2.87267099e-10, 1.33573874e-09]]), array([[426, 307]])),
 5: (array([[3.67804442e-09, 4.59268623e-09]]), array([[184, 426]])),
 6: (array([[4.82346052e-10, 6.49955534e-10]]), array([[307, 426]])),
 7: (array([[1.71544684e-05, 9.29796931e-05]]), array([[330, 448]])),
 8: (array([[8.04776940e-06, 8.04877895e-06]]), array([[426, 307]])),
 9: (array([[5.59139846e-10, 1.99393035e-09]]), array([[426, 307]])),
 10: (array([[5.37392353e-11, 6.49186926e-10]]), array([[426, 307]])),
 11: (array([[2.76300129e-05, 8.14040290e-05]]), array([[280, 370]])),
 12: (array([[3.21775406e-09, 3.48489826e-09]]), array([[426, 184]])),
 13: (array([[1.48803114e-09, 2.00849259e-09]]), array([[426, 307]])),
 14: (array([[1.

In [22]:
list_size = len(neighbors_list)
num_neighbors = len(neighbors_list[0][1][0])

neighbors_idx_array = neighbors_list[0][1][0]
neighbors_distance_array = neighbors_list[0][0][0]
for line in range(1,list_size):
    neighbors_idx_array = np.concatenate((neighbors_idx_array, neighbors_list[line][1][0]), axis=None) 
    neighbors_distance_array = np.concatenate((neighbors_distance_array, neighbors_list[line][0][0]), axis=None) 

if len(neighbors_idx_array) != list_size*num_neighbors:
    print("ERROR: Check array size.")
# Temos agora um array unidimensional com os índices dos clientes recomendados

In [61]:
neighbors_distance_array[0]

2.046642855191294e-10

Criar um dicionário com o número da linha, id e distância

In [24]:
dicio = {}
for idx,ind in zip(neighbors_idx_array, range(len(neighbors_idx_array))):
    dicio[ind] = (idx, market_sample.iloc[idx].name, neighbors_distance_array[ind])

In [25]:
dicio

{0: (307,
  'be75954872c8bb4d3b30d8a99f386a212ec4472b69927748bca0acbe128aefae',
  2.046642855191294e-10),
 1: (426,
  '62add45f81b6da4afd2669b135e761e3d567e1270557db78e13bf8d65dc25a00',
  9.291828417801185e-10),
 2: (307,
  'be75954872c8bb4d3b30d8a99f386a212ec4472b69927748bca0acbe128aefae',
  1.970278384888502e-10),
 3: (426,
  '62add45f81b6da4afd2669b135e761e3d567e1270557db78e13bf8d65dc25a00',
  8.111273874789049e-10),
 4: (426,
  '62add45f81b6da4afd2669b135e761e3d567e1270557db78e13bf8d65dc25a00',
  2.8651403471968706e-10),
 5: (307,
  'be75954872c8bb4d3b30d8a99f386a212ec4472b69927748bca0acbe128aefae',
  1.3379518604139662e-09),
 6: (307,
  'be75954872c8bb4d3b30d8a99f386a212ec4472b69927748bca0acbe128aefae',
  2.0511259357647305e-10),
 7: (426,
  '62add45f81b6da4afd2669b135e761e3d567e1270557db78e13bf8d65dc25a00',
  8.61469340307508e-10),
 8: (426,
  '62add45f81b6da4afd2669b135e761e3d567e1270557db78e13bf8d65dc25a00',
  2.872670989972903e-10),
 9: (307,
  'be75954872c8bb4d3b30d8a99f386a2

In [26]:
neig_df = pd.DataFrame.from_dict(dicio,orient='index')
neig_df.rename(columns={0:'row', 1:'id',2:'distance'},inplace = True)
neig_df.set_index('id', inplace=True)
#neig_df.row.value_counts()

In [27]:
conflicts = neig_df.merge(port1, how='inner',left_index=True,right_index=True)
suggestion_with_conflicts = neig_df.merge(market, how='left',left_index=True,right_index=True)
#neig_df.shape, port1.shape, conflicts.shape, suggestion_with_conflicts.shape, 
suggestions = suggestion_with_conflicts.drop(conflicts.index)

In [28]:
suggestions

,row,distance,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_empresa_anos,idade_emp_cat,...,nm_meso_regiao,nm_micro_regiao,fl_passivel_iss,qt_socios,idade_media_socios,qt_socios_feminino,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo,vl_faturamento_estimado_aux,vl_faturamento_estimado_grupo_aux
id,,,,,,,,,,,,,,,,,,,,,
05e3bfff836590ef9d4bb186c39ae43bdaf244eea0eaf89a35f86046d35b55d9,442,6.247437e-05,0,EMPRESARIO INDIVIDUAL,PI,OUTROS,COMERCIO VAREJISTA,COMERCIO,12.608219,10 a 15,...,CENTRO NORTE PIAUIENSE,TERESINA,0,0.0,0.0,0.0,"DE R$ 81.000,01 A R$ 360.000,00","DE R$ 360.000,01 A R$ 1.500.000,00",210000.0,395457.6
05e3bfff836590ef9d4bb186c39ae43bdaf244eea0eaf89a35f86046d35b55d9,442,8.794085e-05,0,EMPRESARIO INDIVIDUAL,PI,OUTROS,COMERCIO VAREJISTA,COMERCIO,12.608219,10 a 15,...,CENTRO NORTE PIAUIENSE,TERESINA,0,0.0,0.0,0.0,"DE R$ 81.000,01 A R$ 360.000,00","DE R$ 360.000,01 A R$ 1.500.000,00",210000.0,395457.6
05e3bfff836590ef9d4bb186c39ae43bdaf244eea0eaf89a35f86046d35b55d9,442,1.212365e-04,0,EMPRESARIO INDIVIDUAL,PI,OUTROS,COMERCIO VAREJISTA,COMERCIO,12.608219,10 a 15,...,CENTRO NORTE PIAUIENSE,TERESINA,0,0.0,0.0,0.0,"DE R$ 81.000,01 A R$ 360.000,00","DE R$ 360.000,01 A R$ 1.500.000,00",210000.0,395457.6
05e3bfff836590ef9d4bb186c39ae43bdaf244eea0eaf89a35f86046d35b55d9,442,3.613141e-04,0,EMPRESARIO INDIVIDUAL,PI,OUTROS,COMERCIO VAREJISTA,COMERCIO,12.608219,10 a 15,...,CENTRO NORTE PIAUIENSE,TERESINA,0,0.0,0.0,0.0,"DE R$ 81.000,01 A R$ 360.000,00","DE R$ 360.000,01 A R$ 1.500.000,00",210000.0,395457.6
05e3bfff836590ef9d4bb186c39ae43bdaf244eea0eaf89a35f86046d35b55d9,442,6.696316e-05,0,EMPRESARIO INDIVIDUAL,PI,OUTROS,COMERCIO VAREJISTA,COMERCIO,12.608219,10 a 15,...,CENTRO NORTE PIAUIENSE,TERESINA,0,0.0,0.0,0.0,"DE R$ 81.000,01 A R$ 360.000,00","DE R$ 360.000,01 A R$ 1.500.000,00",210000.0,395457.6
11904c55ec4ca8c4cd485f60ccc79756bb0d58423cd1b261e841f1a3310bc0ec,61,1.258684e-04,1,SOCIEDADE EMPRESARIA LIMITADA,RO,ENTIDADES EMPRESARIAIS,COMERCIO POR ATACADO,COMERCIO,15.484932,15 a 20,...,other,other,1,2.0,46.5,0.0,"DE R$ 1.500.000,01 A R$ 4.800.000,00","DE R$ 1.500.000,01 A R$ 4.800.000,00",1607299.2,1817299.2
11904c55ec4ca8c4cd485f60ccc79756bb0d58423cd1b261e841f1a3310bc0ec,61,1.529580e-04,1,SOCIEDADE EMPRESARIA LIMITADA,RO,ENTIDADES EMPRESARIAIS,COMERCIO POR ATACADO,COMERCIO,15.484932,15 a 20,...,other,other,1,2.0,46.5,0.0,"DE R$ 1.500.000,01 A R$ 4.800.000,00","DE R$ 1.500.000,01 A R$ 4.800.000,00",1607299.2,1817299.2
11904c55ec4ca8c4cd485f60ccc79756bb0d58423cd1b261e841f1a3310bc0ec,61,3.365907e-04,1,SOCIEDADE EMPRESARIA LIMITADA,RO,ENTIDADES EMPRESARIAIS,COMERCIO POR ATACADO,COMERCIO,15.484932,15 a 20,...,other,other,1,2.0,46.5,0.0,"DE R$ 1.500.000,01 A R$ 4.800.000,00","DE R$ 1.500.000,01 A R$ 4.800.000,00",1607299.2,1817299.2
11904c55ec4ca8c4cd485f60ccc79756bb0d58423cd1b261e841f1a3310bc0ec,61,1.901273e-04,1,SOCIEDADE EMPRESARIA LIMITADA,RO,ENTIDADES EMPRESARIAIS,COMERCIO POR ATACADO,COMERCIO,15.484932,15 a 20,...,other,other,1,2.0,46.5,0.0,"DE R$ 1.500.000,01 A R$ 4.800.000,00","DE R$ 1.500.000,01 A R$ 4.800.000,00",1607299.2,1817299.2


# Visualização dos resultados

## Mapa (Heat)

### Carrega arquivo com informações geográficas
Gerado por `analysis/Save_IBGE_data_for_Map_Visualization_in_Recommendation_System.ipynb`

In [136]:
coord_micro = pd.read_csv('coord_geograficas_microrregioes_brasil.csv')

In [137]:
### Importa módulo Folium
import folium
from folium import plugins
import random

### Gera lista com coordenadas das sugestões (suggestions)

In [138]:
map_coord = []
null_count = 0  # para verificar quantas sugestões não tem informação da microrregião
for local in suggestions['nm_micro_regiao']:
    coord = coord_micro[coord_micro['nm_micro'] == local][['lat', 'lng']]
    if not coord.empty:
        map_coord.append([coord['lat'].values[0]-random.uniform(0, 0.25),
                          coord['lng'].values[0]-random.uniform(0, 0.25)])
    else:
        null_count = null_count + 1
        # print('Location null', null_count)   #TODO: mostrar como warning
print('Suggestions without info about nm_micro:', null_count)

Suggestions without info about nm_micro: 93


### Mostra heat map

In [139]:
mapa = folium.Map(location=[-8.788497,-53.879873],tiles='Stamen Toner',zoom_start=5)

mapa.add_child(plugins.HeatMap(map_coord))
mapa

## Mapa com marcadores
Passe o mouse sobre os marcadores para informação adicional.

In [140]:
# Para não ter repetições no mapa com marcadores, vou criar um novo df a partir de suggestions,
# deixando só uma entrada para cada id do market
unique_suggestions = pd.DataFrame()
for client in suggestions['row'].unique():
    idxmin_distance = suggestions[suggestions['row']==client]['distance'].idxmin()
    unique_suggestions = unique_suggestions.append(market.loc[idxmin_distance])
unique_suggestions

,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo,de_natureza_juridica,de_nivel_atividade,de_ramo,de_saude_rescencia,de_saude_tributaria,dt_situacao,empsetorcensitariofaixarendapopulacao,fl_email,...,nm_micro_regiao,nm_segmento,nu_meses_rescencia,qt_socios,qt_socios_feminino,setor,sg_uf,sg_uf_matriz,vl_faturamento_estimado_aux,vl_faturamento_estimado_grupo_aux
05e3bfff836590ef9d4bb186c39ae43bdaf244eea0eaf89a35f86046d35b55d9,"DE R$ 81.000,01 A R$ 360.000,00","DE R$ 360.000,01 A R$ 1.500.000,00",EMPRESARIO INDIVIDUAL,MEDIA,COMERCIO VAREJISTA,ACIMA DE 1 ANO,VERDE,2006-03-23,582.96,0.0,...,TERESINA,COMERCIO; REPARACAO DE VEICULOS AUTOMOTORES E ...,22.0,0.0,0.0,COMERCIO,PI,PI,210000.0,395457.6
11904c55ec4ca8c4cd485f60ccc79756bb0d58423cd1b261e841f1a3310bc0ec,"DE R$ 1.500.000,01 A R$ 4.800.000,00","DE R$ 1.500.000,01 A R$ 4.800.000,00",SOCIEDADE EMPRESARIA LIMITADA,ALTA,COMERCIO POR ATACADO,ACIMA DE 1 ANO,AZUL,2003-05-08,0.00,0.0,...,other,COMERCIO; REPARACAO DE VEICULOS AUTOMOTORES E ...,48.0,2.0,0.0,COMERCIO,RO,RO,1607299.2,1817299.2
1822ecaca95aac2d0473bbb5bc87b3d4cdbd141cd9b6e1acce065adc614441aa,"DE R$ 81.000,01 A R$ 360.000,00","DE R$ 1.500.000,01 A R$ 4.800.000,00",EMPRESARIO INDIVIDUAL,MEDIA,SERVICOS DE ALOJAMENTO/ALIMENTACAO,ACIMA DE 1 ANO,VERDE,2007-03-29,1404.58,0.0,...,CODO,ALOJAMENTO E ALIMENTACAO,23.0,1.0,0.0,SERVIÇO,MA,MA,210000.0,4598237.0
1d68cf548046cf294564c8283528d47d32a840d79ab62c4b87d7f7c37e45706d,"DE R$ 1.500.000,01 A R$ 4.800.000,00","DE R$ 10.000.000,01 A R$ 30.000.000,00",SOCIEDADE EMPRESARIA LIMITADA,ALTA,COMERCIO POR ATACADO,ACIMA DE 1 ANO,VERDE,2005-11-03,4177.69,0.0,...,NATAL,COMERCIO; REPARACAO DE VEICULOS AUTOMOTORES E ...,24.0,2.0,0.0,COMERCIO,RN,PE,2110000.0,13719275.0
2ab0f89a246c49862dfc09ec99db369cbc2783b453deb7adc10d035358a4d6ba,"DE R$ 4.800.000,01 A R$ 10.000.000,00","DE R$ 4.800.000,01 A R$ 10.000.000,00",EMPRESA INDIVIDUAL DE RESPONSABILIDADE LIMITAD...,ALTA,BENS DE CONSUMO,ACIMA DE 1 ANO,LARANJA,2007-10-08,1405.65,0.0,...,other,INDUSTRIAS DE TRANSFORMACAO,23.0,1.0,0.0,INDUSTRIA,RO,RO,8345592.0,8345592.0
2dd38379e655d167024910b1077d84682d30cdddd7eb558cb8a951f21976adcd,"DE R$ 1.500.000,01 A R$ 4.800.000,00","DE R$ 1.500.000,01 A R$ 4.800.000,00",SOCIEDADE EMPRESARIA LIMITADA,ALTA,INDUSTRIA DA CONSTRUCAO,ACIMA DE 1 ANO,AZUL,2010-09-17,4268.09,1.0,...,NATAL,CONSTRUCAO,23.0,2.0,0.0,CONSTRUÇÃO CIVIL,RN,RN,1689724.8,2529724.8
30e4fb558e31c472f5134853bc89576443f325f15d4c66220cebff61ddddd677,"DE R$ 81.000,01 A R$ 360.000,00","DE R$ 360.000,01 A R$ 1.500.000,00",EMPRESARIO INDIVIDUAL,ALTA,"TRANSPORTE, ARMAZENAGEM E CORREIO",ACIMA DE 1 ANO,VERDE,2009-09-30,1190.45,1.0,...,ANGICOS,TRANSPORTE ARMAZENAGEM E CORREIO,22.0,1.0,0.0,SERVIÇO,RN,RN,210000.0,753638.4
3a18ce38828adcd19a1e8e60bd34c79bf51a90ef72c0c12276f0d8dffde5e82f,"DE R$ 81.000,01 A R$ 360.000,00","DE R$ 360.000,01 A R$ 1.500.000,00",SOCIEDADE EMPRESARIA LIMITADA,ALTA,SERVICOS ADMINISTRATIVOS,ACIMA DE 1 ANO,VERDE,2005-11-03,555.90,0.0,...,PICOS,ATIVIDADES ADMINISTRATIVAS E SERVICOS COMPLEME...,21.0,0.0,0.0,SERVIÇO,PI,PI,210000.0,1173638.4
3bc22585410624543970edbb68aa5b509f8077f112c5517d1395cdd20bac35fb,"DE R$ 10.000.000,01 A R$ 30.000.000,00","DE R$ 10.000.000,01 A R$ 30.000.000,00",SOCIEDADE EMPRESARIA LIMITADA,ALTA,SERVICOS DE EDUCACAO,ACIMA DE 1 ANO,LARANJA,2005-11-03,1315.56,0.0,...,MANAUS,EDUCACAO,48.0,2.0,2.0,SERVIÇO,AM,AM,10869876.0,10869876.0
3dc612e72f3e6e6f4921e33f29cff5e60df3a0a8d6698b91b0079e1af4135ee9,"DE R$ 81.000,01 A R$ 360.000,00","DE R$ 360.000,01 A R$ 1.500.000,00",SOCIEDADE EMPRESARIA LIMITADA,MEDIA,COMERCIO VAREJISTA,ACIMA DE 1 ANO,LARANJA,2005-11-03,0.00,0.0,...,MANAUS,COMERCIO; REPARACAO DE VEICULOS AUTOMOTORES E ...,24.0,2.0,2.0,COMERCIO,AM,AM,210000.0,420000.0


In [141]:
uniq_map_coord = []
null_count = 0  # para verificar quantas sugestões não tem informação da microrregião
for local in zip(unique_suggestions[['nm_micro_regiao', 'de_ramo']].to_numpy(),
                 unique_suggestions[['nm_micro_regiao', 'de_ramo']].index):
    coord = coord_micro[coord_micro['nm_micro'] == local[0][0]][['lat', 'lng']]
    if not coord.empty:
        uniq_map_coord.append([coord['lat'].values[0]-random.uniform(0, 0.5),
                          coord['lng'].values[0]-random.uniform(0, 0.5),
                          'id:'+local[1]+' ramo:'+local[0][1]]) # para o popup com 
                          #informação do id e ramo da empresa de sugestão.
    else:
        null_count = null_count + 1
        # print('Location null', null_count)   #TODO: mostrar como warning

print('Suggestions without info about nm_micro:', null_count)

Suggestions without info about nm_micro: 5


In [135]:
# Mostra marcadores com informação do id e ramo da empresa de sugestão.
markmapa = folium.Map(location=[-8.788497, -53.879873],
                  tiles='OpenStreetMap', zoom_start=5)

#tooltip = 'Click for info'
for mark in uniq_map_coord:
    folium.Marker([mark[0], mark[1]], popup=mark[2], tooltip=mark[2]).add_to(markmapa)
markmapa

## Lista ordenada por faixa de faturamento